In [1]:
import warnings
import numpy as np
import matplotlib.pyplot as plt
from numpy.random import default_rng
import itertools as it
import sys
import progressbar
import time
import random

from joblib import Parallel, delayed
from joblib import Parallel, delayed

from scipy import linalg
from scipy.linalg import LinAlgWarning

import pickle

warnings.filterwarnings(action='error', category=LinAlgWarning, module='sklearn')

def stack_xy(A):
    return np.block([[A["xx"], A["xy"]], 
                    [A["xy"], A["yy"]]])

def restructure_dict(list_dic, shape):
    keys = list( list_dic[0].keys() )

    new_dic = dict()
    
    for key in keys:
        new_dic[key] = []
    
    for item in list_dic:
        for key in keys:
            new_dic[key].append(item[key]) 

    for key in keys:
        arr = np.asarray(new_dic[key])
        new_dic[key] = arr.reshape(shape)
        
    return new_dic

In [2]:
def create_G(rx, ry=0., rz=0., k=2.*np.pi, thres_same=0.1, thres_ren=1e-12):
    
    r_in = np.sqrt(rx**2 + ry**2 + rz**2) # input r
    
    rx_ren = rx + thres_ren*np.random.random_sample(r_in.shape)    
    r = np.sqrt(rx_ren**2 + ry**2 + rz**2)  # add some random noise to renormalize for divergences

    
    phase = np.exp(1j*k*r)/(4*np.pi*r)

    G = dict()
    G["xx"] = phase * ( 1. + (1j*k*r-1.)/(k*r)**2 + ( -1. + (3.-3.*1j*k*r)/(k*r)**2) * rx**2/r**2 ) 
    G["yy"] = phase * ( 1. + (1j*k*r-1.)/(k*r)**2 + ( -1. + (3.-3.*1j*k*r)/(k*r)**2) * ry**2/r**2 )
    G["xy"] = phase * ( ( -1. + (3.-3.*1j*k*r)/(k*r)**2) * rx*ry/r**2 )
    
    for key in G.keys():
        G[key][r_in<thres_same] = 0. # set vals with r=0. to zero (don't count emitter interacting with itself)

    return G

def create_G_y_average(rx, y_max, N_y, thres_same=0.1):
    
    N_x = rx.shape[0]
    keys = ['xx', 'yy', 'xy']
    v_y = np.linspace(0., y_max, N_y)
    
    G_mean = create_G(rx, ry=0., thres_same=thres_same)
    for y in v_y[1:]:
        Gp = create_G(rx, ry=y, thres_same=thres_same)
        Gm = create_G(rx, ry=-y, thres_same=thres_same)
        
        for key in keys:
            G_mean[key] += Gp[key] + Gm[key]
            
    for key in keys:
        G_mean[key] = G_mean[key] *2.*y_max/ (2*N_y-1)
    
    return G_mean
            
    

def create_alpha_array(delta, d_delta, a, r, gamma_0, gamma_nr=0., lambda_a=1., epsilon=12.25):
    gamma = gamma_0 + gamma_nr
    delta_r = delta + d_delta*np.sin(2.*np.pi*r/a)
    return -3./(4.*np.pi**2)*epsilon*lambda_a**3*(gamma_0/2.)/(delta_r+1j*gamma/2.)


In [3]:
def get_sample_1D(a, delta, gamma_0, gamma_nr, n_exc, G, d_r, Gaussian=False, d_delta=1., epsilon=12.25):
        
    # beam profile 
    if Gaussian:
        n_0 = n_exc * np.exp(-(v_r-v_r[N_r//2])**2/(2.*width**2))
    else:
        n_0 = n_exc * np.ones(N_r)
    
    # get polarizations
    v_alpha = create_alpha_array(delta, d_delta, a, v_r, gamma_0, gamma_nr=gamma_nr, epsilon=epsilon)
    p_0 = n_0 * v_alpha

    # solve equations to find effective polarizations
    p = dict()
    p['x'] = linalg.solve( np.eye(N_r) - 4.*np.pi**2/epsilon * d_r *(np.diag(v_alpha)@G["xx"]), p_0)
    p['y'] = linalg.solve( np.eye(N_r) - 4.*np.pi**2/epsilon * d_r *(np.diag(v_alpha)@G["yy"]), p_0)
    p['bare'] = p_0
    
    return p

def get_sample(a, delta, gamma_0, gamma_nr, n_exc, G, d_x, d_delta=1., Gaussian=None, epsilon=12.25, only_1D=False):
        
    # beam profile 
    if Gaussian is not None:
        n_0 = n_exc * np.exp(-(v_r-v_r[N_r//2])**2/(2.*Gaussian['width']**2))
    else:
        n_0 = n_exc * np.ones(N_r)
        
    # get dyadic Green's function with correct cutoff
    r_ij = v_r[:,None] - v_r[None,:]
    G_dict = create_G(r_ij, thres_same=r_excl)
    G_full = stack_xy(G)
    d_r = v_r[1] - v_r[0]
    
    
    
    # get polarizations
    v_alpha = create_alpha_array(delta, d_delta, a, v_r, gamma_0, gamma_nr=gamma_nr, epsilon=epsilon)
    p_0 = n_0 * v_alpha
    p = dict()
    if only_1D:
        # solve equations to find effective polarizations
        p['x'] = linalg.solve( np.eye(N_r) - 4.*np.pi**2/epsilon * d_r *(np.diag(v_alpha)@G["xx"]), p_0)
        p['y'] = linalg.solve( np.eye(N_r) - 4.*np.pi**2/epsilon * d_r *(np.diag(v_alpha)@G["yy"]), p_0)
        p['bare'] = p_0
        
    else:
        Gd = stack_xy(G)
        pd_0 = np.concatenate((p_0,p_0))
        alphad = np.concatenate((v_alpha,v_alpha))
        pd = linalg.solve( np.eye(2*N_r) - 4.*np.pi**2/epsilon * d_r *(np.diag(alphad)@Gd), pd_0)
        p['x'] = pd[:N_r]
        p['y'] = pd[N_r:]
        p['bare'] = p_0
    
    return p

## generate data (a,\Delta\omega, delta)

In [4]:
# system parameters (characterizing data set)
gamma_0 = 1. # radiative losses
gamma_nr = 0. # non-radiative losses
n_exc = 10. # exciton density (units 1/\lambda)

r_max = 10. # for simulation maximal r sample
N_r = 400 # number of points for discretization

y_average = True
y_max = r_max / 2.
N_y = N_r

# modulated exciton shift amplitude
D_min = 0.
D_max = 10.
N_D = 21

# delta to evaluate to obtain
delta_min = -2.*D_max # detuning from bare resonance
delta_max = 2.*D_max
N_delta = 100

# for the paralel loop
a_min = 1e-6 # a's to scan
a_max = 2.
N_a = 41


########################################

v_r = np.linspace(0., r_max, N_r)
r_ij = np.abs(v_r[:,None] - v_r[None,:])
d_r = v_r[1] - v_r[0]
r_excl = 0.5/n_exc

if y_average:
    G = create_G_y_average(r_ij, y_max, N_y, thres_same=r_excl)
else:
    G = create_G(r_ij, thres_same=r_excl)


v_D = np.linspace(D_min, D_max, N_D)
v_a = np.linspace(a_min, a_max, N_a)
v_delta = np.linspace(delta_min, delta_max, N_delta)

# parallel loop
data_pp = Parallel(n_jobs=-1)(delayed(get_sample)
                                      (a, delta, gamma_0, gamma_nr, n_exc, G, d_r, d_delta=D) 
                                      for a in v_a for D in v_D for delta in v_delta)


In [5]:
# save everything

folder = "store"
filename = "/new_gamma0_{:.2f}_gammanr_{:.2f}_nexc_{:.2f}_rmax_{:.2f}".format(gamma_0, gamma_nr, n_exc, r_max)

# restructure result (bring keys outward)
data = restructure_dict(data_pp, shape=(N_a,N_D,N_delta,N_r))

# put some other things in dict, useful for later
data["a"] = v_a
data["D"] = v_D
data["delta"] = v_delta
data["r"] = v_r

with open(folder + filename + '.pkl', 'wb') as f:
      pickle.dump(data, f, pickle.HIGHEST_PROTOCOL)

In [14]:
n_exc

10.0